In [7]:
import nest_asyncio
nest_asyncio.apply()

SyntaxError: invalid syntax (<ipython-input-7-2b8b366b108f>, line 1)

In [8]:
from llama_index.core import SimpleDirectoryReader

In [9]:
document = SimpleDirectoryReader(input_files=['/content/final_doc.pdf']).load_data()

In [19]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = splitter.get_nodes_from_documents(document)

In [21]:
node_metadata = nodes[10].get_content(metadata_mode=True)
print(node_metadata)

page_label: 6
file_name: final_doc.pdf
file_path: /content/final_doc.pdf
file_type: application/pdf
file_size: 28772791
creation_date: 2024-06-10
last_modified_date: 2024-06-10

information
On 1996-06-10 00:00:00, Netherlands played against Scotland in Birmingham, England. The match was a UEFA Euro and was
not played on neutral ground. The home team Netherlands scored 0 goals, while the away team Scotland scored 0 goals. 
On 1996-06-10 00:00:00, Romania played against France in Newcastle, England. The match was a UEFA Euro and was not
played on neutral ground. The home team Romania scored 0 goals, while the away team France scored 1 goals. 
On 1996-06-11 00:00:00, Turkey played against Croatia in Nottingham, England. The match was a UEFA Euro and was not
played on neutral ground. The home team Turkey scored 0 goals, while the away team Croatia scored 1 goals. 
On 1996-06-13 00:00:00, Switzerland played against Netherlands in Birmingham, England. The match was a UEFA Euro and
was not pl

In [34]:

import os
from typing import List, Optional

from llama_index.llms.huggingface import (
    HuggingFaceInferenceAPI,
    HuggingFaceLLM,
)

In [35]:
HF_TOKEN: Optional[str] = os.getenv("HUGGING_FACE_TOKEN")

In [43]:
from llama_index.core import Settings
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [45]:
Settings.llm = HuggingFaceLLM(model_name="microsoft/Phi-3-medium-128k-instruct")
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [48]:
from llama_index.core import SummaryIndex, VectorStoreIndex
summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [51]:
summary_query_engine = summary_index.as_query_engine(
    response_mode = "tree_summarize",
    use_async = True
)

vector_query_engine = vector_index.as_query_engine()

In [52]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to the Lora paper."
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the the Lora paper."
    ),
)

In [53]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [57]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

IndexError: index out of range in self